# Grab all the loopy/inlining/odd shapes
Select a couple of examples to see what the pattern is for `stop_sequence` and `shape_meters`.

In [1]:
import os
os.environ['USE_PYGEOS'] = '0'

import dask.dataframe as dd
import geopandas as gpd
import pandas as pd
import numpy as np
import shapely
import sys

from segment_speed_utils import (helpers, gtfs_schedule_wrangling, 
                                 wrangle_shapes)
from segment_speed_utils.project_vars import SEGMENT_GCS, analysis_date

# Temp append so we can import functions from scripts/
sys.path.append("./scripts/")
import prep_stop_segments
import cut_stop_segments
import loop_utils

shape_with_utc = "f765b9d12fcca0173b4e3ddbc0374d18"
another_shape = "007cf76062f5957d4e38ea54e624c0ad"

In [2]:
loop_shapes = loop_utils.grab_loop_shapes(analysis_date)

stop_times_with_geom = prep_stop_segments.stop_times_aggregated_to_shape_array_key(
        analysis_date, loop_shapes)

st_loops = stop_times_with_geom.compute()

In [3]:
gdf = (loop_utils.assign_visits_to_stop(st_loops)
       .sort_values(["shape_array_key", "stop_sequence"])
       .reset_index(drop=True)
      )

In [4]:
gdf2 = gdf[gdf.shape_array_key.isin([another_shape])]

In [60]:
def get_index(array: np.ndarray, item) -> int:
    """
    """
    #https://www.geeksforgeeks.org/how-to-find-the-index-of-value-in-numpy-array/
    for idx, val in np.ndenumerate(array):
        if val == item:
            return idx[0]
        
        
def include_prior_and_subsequent(
    array: np.ndarray, value: int
) -> np.ndarray:
    idx = get_index(array, value)
    
    subset_array = array[idx-1: idx+2]
    
    # For the first stop sequence, there is no prior, 
    # but the result of that is grabbing nothing
    if len(subset_array) == 0:
        subset_array  = array[idx: idx+2]
        
    # for the last stop sequence, there is no subsequent,
    # and the subset of array grabs 2 items, which is ok        
    
    return subset_array

In [65]:
test = include_prior_and_subsequent(stop_seq_array, 33)
test

array([32, 33])

In [6]:
shape_geom = gdf2.geometry.iloc[0]
stop_geom_array = np.array(gdf2.stop_geometry)
stop_seq_array = np.array(gdf2.stop_sequence)

In [7]:
# For a stop sequence value, find the prior and subsequent stop sequence
# this is important especially because stop_sequence does not have 
# to be increasing in increments of 1, but it has to be monotonically increasing
stop_seq = 15
subset_seq = include_prior_and_subsequent(stop_seq_array, stop_seq)
subset_seq

array([14, 15, 16])

In [43]:
# Grab relevant subset based on stop sequence values to get stop geometry subset
#https://stackoverflow.com/questions/5508352/indexing-numpy-array-with-another-numpy-array
subset_stop_geom = stop_geom_array[subset_seq]

In [67]:
def get_direction_vector(
    start: shapely.geometry.Point, 
    end: shapely.geometry.Point
) -> tuple:
    """
    Given 2 points (in a projected CRS...not WGS84), return a 
    tuple that shows (delta_x, delta_y).

    https://www.varsitytutors.com/precalculus-help/find-a-direction-vector-when-given-two-points
    """
    return ((end.x - start.x), (end.y - start.y))
    
    
def distill_array_into_direction_vector(array: np.ndarray) -> tuple:
    """
    Given an array of n items, let's take the start/end of that.
    From start/end, we can turn 2 coordinate points into 1 distance vector.
    Distance vector is a tuple that equals (delta_x, delta_y).
    """
    return get_direction_vector(array[0], array[-1])


In [68]:
def get_vector_norm(vector: tuple) -> float:
    """
    Get the length (off of Pythagorean Theorem) by summing up
    the squares of the components and then taking square root.
    
    Use Pythagorean Theorem to get unit vector. Divide the vector 
    by the length of the vector to get unit/normalized vector.
    This equation tells us what we need to divide by.
    """
    return np.sqrt(vector[0]**2 + vector[1]**2)


def get_normalized_vector(vector: tuple) -> tuple:
    """
    Apply Pythagorean Theorem and normalize the vector of distances.
    """
    x_norm = vector[0] / get_vector_norm(vector)
    y_norm = vector[1] / get_vector_norm(vector)

    return (x_norm, y_norm)


def dot_product(vec1: tuple, vec2: tuple) -> float:
    """
    Take the dot product. Multiply the x components, the y components, and 
    sum it up.
    """
    return vec1[0]*vec2[0] + vec1[1]*vec2[1]

In [69]:
# Take current stop
# find stop_sequence before and after

## NEED THIS
#stop_geom_array = np.array(gdf2.stop_geometry)
#stop_seq_array = np.array(gdf2.stop_sequence)

def super_project(
    current_stop_seq: int,
    shape_geometry: shapely.geometry.LineString,
    stop_geometry_array: np.ndarray,
    stop_sequence_array: np.ndarray,
):
    
    # Set up: project each of the coordinates in the shape_geometry
    # to be shape_meters
    shape_dist_array = np.array(
            [shape_geometry.project(shapely.geometry.Point(p)) 
            for p in shape_geometry.coords]
        )
    
    # (1) Given a stop sequence value, find the stop_sequence values 
    # just flanking it (prior and subsequent).
    subset_seq = include_prior_and_subsequent(
        stop_sequence_array, current_stop_seq)

    
    # (2) Grab relevant subset based on stop sequence values to get stop geometry subset
    # https://stackoverflow.com/questions/5508352/indexing-numpy-array-with-another-numpy-array
    subset_stop_geom = stop_geometry_array[subset_seq]

    # (3) Project this vector of 3 stops
    # because we need to know which part to subset
    # off of the shape's shape_meters array
    subset_stop_proj = np.array(
        [shape_geometry.project(i) for i in subset_stop_geom]
    )

    # (4) Get the subset of shape_path points that
    # spans a start distance position and end distance position 
    # We have 2 stops, and let's grab the chunk of the shape that spans that
    # https://stackoverflow.com/questions/16343752/numpy-where-function-multiple-conditions

    # the start_dist and end_dist take the prior/subsequent stop
    # to use to check against direction
    shape_dist_subset = shape_dist_array[
        (np.where(
            (shape_dist_array >= subset_stop_proj[0]) & 
            (shape_dist_array <= subset_stop_proj[-1]))
        )]
    
    # Interpolate again so we change the shape_meters back into coordinate points
    subset_shape_geom = [shape_geometry.interpolate(i) 
                           for i in shape_dist_subset]

    # If the stop vector and shape vector run in the same direction,
    # then we want to cut a segment from prior to current stop.
    dot_prod = find_if_two_arrays_same_direction(
        subset_stop_geom,
        subset_shape_geom
    )
    
    



In [70]:
def find_if_two_arrays_same_direction(
    subset_stop_geom_array: np.ndarray, 
    subset_shape_geom_array: np.ndarray
) -> float:
    """
    """
    # Get vectors for stop and shape
    stop_vec = distill_array_into_direction_vector(subset_stop_geom_array)
    shape_vec = distill_array_into_direction_vector(subset_shape_geom_array)
    
    # Normalize the vectors (divide by length) to get unit vector
    stop_norm = get_normalized_vector(stop_vec)
    shape_norm = get_normalized_vector(shape_vec)
    
    dot_result = dot_product(stop_norm, shape_norm)
    
    return dot_result

In [71]:
start_stop = subset_stop_geom[0]
end_stop = subset_stop_geom[-1]

stop_vec = get_direction_vector(start_stop, end_stop)

In [72]:
# Now project this vector of 3 stops
# because we need to know which part to subset
# off of the shape's shape_meters array
subset_stop_projected = np.array(
    [shape_geom.project(i) for i in subset_stop_geom]
)
subset_stop_projected

/opt/conda/lib/python3.9/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point


array([5503.02579871, 5877.33959251, 6007.87823823])

In [18]:
shape_path_dist = np.array(
            [shape_geom.project(shapely.geometry.Point(p)) 
            for p in shape_geom.coords]
)

shape_path_dist[:5]

array([  0.        ,  14.50702629, 100.6041139 , 126.5169287 ,
       126.5169287 ])

In [19]:
# Get the subset of shape_path points that
# covers start_dist to end_dist
# https://stackoverflow.com/questions/16343752/numpy-where-function-multiple-conditions

# the start_dist and end_dist take the prior/subsequent stop
# to use to check against direction
shape_path_subset = shape_path_dist[
    (np.where(
        (shape_path_dist >= subset_stop_projected[0]) & 
        (shape_path_dist <= subset_stop_projected[-1]))
    )]

shape_path_subset

array([5503.17692096, 5503.17692096, 5517.81052203, 5533.71996806,
       5549.33373331, 5582.26940738, 5603.59423777, 5664.84485808,
       5848.62988146, 5859.68089117, 5877.38668559, 5877.38668559,
       5907.37737327])

In [41]:
shape_path_interp = [shape_geom.interpolate(i) 
                     for i in shape_path_subset]

In [21]:
# take the shape's vector and take the start/end of that
# and create vector
# get a vector between the subsequent and prior stops (we draw 
# right through our stop of interest)
start_shape = shape_path_interp[0]
end_shape = shape_path_interp[-1]

shape_vec = get_direction_vector(start_shape, end_shape)
shape_vec

(-387.91065980354324, -96.21564955078065)

In [23]:
#https://stackoverflow.com/questions/17332759/finding-vectors-with-2-points
# normalize to unit vector
# https://stackoverflow.com/questions/21030391/how-to-normalize-a-numpy-array-to-a-unit-vector
stop_vec, shape_vec

((-490.6085929343826, -93.5817481954582),
 (-387.91065980354324, -96.21564955078065))

In [38]:
stop_norm = get_normalized_vector(stop_vec)
shape_norm = get_normalized_vector(shape_vec)

stop_norm, shape_norm

((-0.9822897736499571, -0.18736808848551603),
 (-0.9705894605575428, -0.24074072994534598))

In [73]:
dot_prod = find_if_two_arrays_same_direction(
    subset_stop_geom,
    shape_path_interp
)

dot_prod

0.9985072319085699

In [ ]:
if dot_prod > 0:
    shape_path_interp

In [77]:
use_these_stops = subset_stop_geom[:2]

In [78]:
# Now project this vector of 3 stops
# because we need to know which part to subset
# off of the shape's shape_meters array
use_these_stops_proj = np.array(
    [shape_geom.project(i) for i in use_these_stops]
)
use_these_stops_proj

array([5503.02579871, 5877.33959251])

In [82]:
use_me_shape = shape_path_dist[
    (np.where(
        (shape_path_dist >= use_these_stops_proj[0]) & 
        (shape_path_dist <= use_these_stops_proj[-1]))
    )]

use_me_shape

array([5503.17692096, 5503.17692096, 5517.81052203, 5533.71996806,
       5549.33373331, 5582.26940738, 5603.59423777, 5664.84485808,
       5848.62988146, 5859.68089117])

In [84]:
use_me_shape_interp = [shape_geom.interpolate(i) 
                     for i in use_me_shape]

In [92]:
segment_geom = gpd.GeoSeries(
    shapely.geometry.LineString(use_me_shape_interp), 
    crs="EPSG:3310"
)

In [96]:
use_stops_geom = gpd.GeoSeries(use_these_stops, crs="EPSG:3310")

In [97]:
import folium

In [98]:
m = segment_geom.explore(tiles="CartoDB Positron", name="segment")
m = use_stops_geom.explore(m=m, name="stops")
folium.LayerControl().add_to(m)

In [99]:
m

In [ ]:
 shape_path_dist = np.array(
            [shape_geom.project(shapely.geometry.Point(p)) 
            for p in shape_geom.coords]
        )

In [ ]:
shape_path_dist

In [ ]:
# Turn the stop_geometry and shape_geometry columns into geoseries
shape_geoseries = gpd.GeoSeries(gdf.geometry)
stop_geoseries = gpd.GeoSeries(gdf.stop_geometry)

# Get projected shape_meters as dask array
shape_meters_geoseries = wrangle_shapes.project_point_geom_onto_linestring(
    shape_geoseries,
    stop_geoseries,
    get_dask_array=False
)

# Attach dask array as a column
gdf["shape_meters"] = shape_meters_geoseries

In [ ]:
def add_prior_stop_info(
    gdf: gpd.GeoDataFrame, 
    trip_grouping_cols: list = ["shape_array_key"],
    segment_identifier_cols: list = ["shape_array_key", "stop_sequence"]
):
    # prior stop location won't be available if the first/last stop 
    # is the one being visited twice (which is often the case)
    gdf = gdf.assign(
        prior_stop_location = (gdf.sort_values(segment_identifier_cols)
                               .groupby(trip_grouping_cols)
                               ["stop_geometry"]
                               .apply(lambda x: x.shift(1))
                              ),
        #prior_shape_meters = (gdf.sort_values(segment_identifier_cols)
        #                       .groupby(trip_grouping_cols)
        #                      ["shape_meters"]
        #                       .apply(lambda x: x.shift(1))
        #                     ),
    )
    
    return gdf

In [ ]:
gdf = add_prior_stop_info(
    gdf,
    trip_grouping_cols = ["shape_array_key"],
    segment_identifier_cols = ["shape_array_key", "stop_sequence"]
)

This is a case where there's not a monotonically increasing `shape_meters` column.

But, where it's not monotonically increasing is not only where stops are visited twice. It's also happening at `stop_sequence==2`. 
* `stop_sequence==1` is where the shape begins, and it's also a portion of a loop.
* `stop_sequence==2` is not visited twice, but since it's occurring during the loop, the `shape_meters` calculated is against the end of the shape, not the beginning.
* For a stop's 2nd visit, we want to look at the prior stop and calculate distance, and overwrite `shape_meters` so it **is** monotonically increasing.
* For a stop's 1st visit, we may not be able to look prior (stop 0), and even if we look ahead, stop 2 might have an issue. **But**, if we get the distance between stop 1 and 2, we still might be able to back out the same coordinates from the line geometry, even if the array is not monotonically increasing. The numbers are just our ordered perspective, but maybe for coordinates, it's still able to grab the same ones. 

In [ ]:
display_cols = [
    "shape_array_key", "stop_id", "stop_sequence",
    "num_visits", "visit_order", 
    "shape_meters", 
    "prior_stop_location", 
]
another_shape_gdf = gdf[gdf.shape_array_key==another_shape][
    display_cols + ["geometry", "stop_geometry"]]

another_shape_gdf[["stop_id", "stop_sequence", 
                   "shape_meters"]]

In [ ]:
shape_geom = another_shape_gdf.geometry.iloc[0]
shape_dist_array = np.array([shape_geom.project(shapely.geometry.Point(p)) 
          for p in shape_geom.coords])

In [ ]:
len(shape_dist_array), len(np.unique(shape_dist_array))

In [ ]:
shape_dist_array

In [ ]:
np.unique(shape_dist_array)

In [ ]:
another_shape_gdf.set_geometry("stop_geometry").explore(
    "stop_sequence",
    tiles="CartoDB Positron", 
    categorical=True, legend=False
)

For these shapes, do we want to sort `stop_sequence` instead by `shape_meters`?

Then pick a non-loopy route and see. If we sort and enforce the monotonically increasing rule, what will happen? There was a Sacramento Schedule route that was cut haphazardly.

Look at the array of line geometry coords, and those are also not monotonically increasing. So, it looks like we need to factor in what subset has been grabbed vs not yet (eligible to).

In [ ]:
test_gdf = another_shape_gdf[another_shape_gdf.stop_sequence.isin([1,2])
                             ].reset_index(drop=True)
display(test_gdf[["stop_id", "stop_sequence", "shape_meters"]])

result = cut_stop_segments.get_shape_coords_up_to_stop(
    shape_geom,
    shape_dist_array,
    np.array(test_gdf.shape_meters),
    (test_gdf.shape_meters[0], test_gdf.shape_meters[1])
)

gpd.GeoSeries(result, crs="EPSG:3310").explore(
    tiles="CartoDB Positron")

In [ ]:
stop_geom = gpd.GeoSeries(another_shape_gdf.stop_geometry)
prior_stop_geom = gpd.GeoSeries(another_shape_gdf.prior_stop_location)
distance_from_prior = stop_geom.distance(prior_stop_geom, align=True)

another_shape_gdf["distance_from_prior_stop"] = distance_from_prior

In [ ]:
# Get the cumulative distance 
another_shape_gdf = another_shape_gdf.assign(
    cumulative_dist = (another_shape_gdf.sort_values(
                        ["shape_array_key", "stop_sequence"])
                       .groupby("shape_array_key")
                       .distance_from_prior_stop
                       .cumsum()
                      ),
    shape_meters_adj = (another_shape_gdf[["shape_meters", 
                                           "distance_from_prior_stop"]].sum(axis=1)
                       )
)

In [ ]:
test_gdf = another_shape_gdf[another_shape_gdf.stop_sequence.isin([4,5])
                             ].reset_index(drop=True)
display(test_gdf[["stop_id", "stop_sequence", "shape_meters", 
                  "distance_from_prior_stop",
                  "cumulative_dist", "shape_meters_adj"]])

m = test_gdf.set_geometry("stop_geometry").explore(
    "stop_sequence", tiles='CartoDB Positron'
)

meters_col = "cumulative_dist"

result = cut_stop_segments.get_shape_coords_up_to_stop(
    shape_geom,
    np.unique(shape_dist_array),
    np.array(test_gdf[meters_col]),
    (test_gdf[meters_col][0], test_gdf[meters_col][1])
)

gpd.GeoSeries(result, crs="EPSG:3310").explore(
    tiles="CartoDB Positron", m=m)

In [ ]:
test_gdf = another_shape_gdf[another_shape_gdf.stop_sequence.isin([29,30])
                             ].reset_index(drop=True)
display(test_gdf[["stop_id", "stop_sequence", "shape_meters", 
                  "distance_from_prior_stop",
                  "cumulative_dist", "shape_meters_adj"]])

m = test_gdf.set_geometry("stop_geometry").explore(
    "stop_sequence", tiles='CartoDB Positron'
)

meters_col = "shape_meters_adj"
result = cut_stop_segments.get_shape_coords_up_to_stop(
    shape_geom,
    shape_dist_array,
    np.array(test_gdf[meters_col]),
    (test_gdf[meters_col][0], test_gdf[meters_col][1])
)

gpd.GeoSeries(result, crs="EPSG:3310").explore(
    tiles="CartoDB Positron", m=m)

In [ ]:
test_gdf = another_shape_gdf[another_shape_gdf.stop_sequence.isin([28, 29])
                             ].reset_index(drop=True)
display(test_gdf[["stop_id", "stop_sequence", "shape_meters", 
                 "cumulative_dist", "shape_meters_adj"]])

m = test_gdf.set_geometry("stop_geometry").explore(
    "stop_sequence", tiles='CartoDB Positron'
)

meters_col = "cumulative_dist"

result = cut_stop_segments.get_shape_coords_up_to_stop(
    shape_geom,
    shape_dist_array,
    np.array(test_gdf[meters_col]),
    (test_gdf[meters_col][0], test_gdf[meters_col][1])
)

gpd.GeoSeries(result, crs="EPSG:3310").explore(
    tiles="CartoDB Positron", m=m)

In [ ]:
stop_segments = helpers.import_segments(
    SEGMENT_GCS,
    f"stop_segments_{analysis_date}", 
    filters=[[("shape_array_key", "==", shape_with_utc)],
             [("shape_array_key", "==", another_shape)]
            ]
)

In [ ]:
stop_segments[stop_segments.geometry.notna()
             ].explore("stop_sequence", tiles="CartoDB Positron")